In [1]:
!pip install psycopg2


     |████████████████████████████████| 383 kB 865 kB/s 
  Created wheel for psycopg2: filename=psycopg2-2.8.6-cp37-cp37m-linux_x86_64.whl size=452724 sha256=32c8c20355a55f9e45cba09af22a68adc1f406bb0f1ffa2d6ebd2b9d0f063383
  Stored in directory: /root/.cache/pip/wheels/25/78/75/9c0323f7e1fb42143cbd2439302beb7850a1034abb961cb281
Successfully built psycopg2


In [2]:
import numpy as np
import pandas as pd
import psycopg2


In [3]:
conn = psycopg2.connect("dbname=cricket host =sportsanalytics.in user=postgres password=Sumit@123 ")
series = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4 AND start_date BETWEEN '01-01-2008 00:00:01' AND '01-01-2020 00:00:01'", conn)
query = f"""SELECT id FROM cds_match_match WHERE series_id in {tuple(series['id'])} """
match_ids = pd.read_sql_query(query,conn)
query = f"""SELECT id,match_id FROM cds_scorecard_scorecard WHERE match_id in {tuple(match_ids['id'])} and inning = 1 """
scorecard_ids = pd.read_sql_query(query,conn)
query = f"""SELECT * FROM cds_commentary_commentary WHERE scorecard_id in {tuple(scorecard_ids['id'])} """
data = pd.read_sql_query(query,conn)
data = data[['id', 'ball', 'over', 'total_runs', 'total_wickets','is_wide', 'runs', 'is_out', 'is_no_ball', 'batsmen_id', 'bowler_id', 'offstrike_batsmen_id','player_out_id', 'scorecard_id', 'team_id']]
scorecard_ids.columns = ['scorecard_id', 'match_id']
data = data.merge(scorecard_ids, on='scorecard_id', how='left')
data = data.sort_values(['scorecard_id','over','ball'])

In [4]:
query = f"""SELECT player_id,average,strike_rate FROM cds_playerstats_playerbattingstats WHERE league_id = 4 """
batsman_data = pd.read_sql_query(query,conn)
batsman_data.columns  = ['batsmen_id', 'batsman_average', 'batsman_strike_rate']


In [5]:
query = f"""SELECT player_id,economy,strike_rate FROM cds_playerstats_playerbowlingstats WHERE league_id = 4 """
bowler_data = pd.read_sql_query(query,conn)
bowler_data.columns = ['bowler_id', 'bowling_economy', 'bowler_strike_rate']

In [6]:
def convert_data(data):  
    data = data.merge(batsman_data,on = 'batsmen_id')
    data = data.merge(bowler_data, on = 'bowler_id')
    data = data.query("is_wide == False")
    data = data.query("is_no_ball == False")
    data = data[['ball', 'over', 'total_runs', 'total_wickets', 'runs','is_out', 'batsmen_id', 'bowler_id','offstrike_batsmen_id', 'player_out_id', 'scorecard_id','batsman_average', 'batsman_strike_rate', 'bowling_economy','bowler_strike_rate']]
    my_data = []
    for si in np.unique(np.array(data['scorecard_id'])):
        test = data.query("scorecard_id == {}".format(si)).sort_values('total_runs')
        striker_run=[]
        striker_balls=[]
        sball = {}
        srun = {}
        for i in range(test.shape[0]):
            kamkidata = test[i:i+1]
            if(int(kamkidata['batsmen_id']) in srun.keys()):
                striker_run.append(srun[int(kamkidata['batsmen_id'])])
            else:
                striker_run.append(0)
                srun[int(kamkidata['batsmen_id'])] = 0
        
            if(int(kamkidata['batsmen_id']) in sball.keys()):
                striker_balls.append(sball[int(kamkidata['batsmen_id'])])
            else:
                striker_balls.append(0)
                sball[int(kamkidata['batsmen_id'])] = 0
            sball[int(kamkidata['batsmen_id'])]+=1
            srun[int(kamkidata['batsmen_id'])]+=int(kamkidata['runs'])
        test['striker_runs'] = striker_run
        test['striker_balls'] = striker_balls
        test['ball_number'] = test['ball']+6*test['over']
        test['player_out_id'] = (test['player_out_id']==test['batsmen_id']).astype(int)
        test['is_out'] = test['is_out'].astype(int)
        test = test.drop(columns = ['ball','over','scorecard_id'])
        test['total_runs'] = test['total_runs']-test['runs']
        test['runs_0'] = (test['runs']==0).astype(int)
        test['runs_1'] = (test['runs']==1).astype(int)
        test['runs_2'] = (test['runs']==2).astype(int)
        test['runs_3'] = (test['runs']==3).astype(int)
        test['runs_4'] = (test['runs']==4).astype(int)
        test['runs_6'] = (test['runs']==6).astype(int)
        test = test.drop(columns = ['runs'])
        my_data.append(test)
    final_data = pd.concat(my_data)
    input_data= np.array(final_data[['total_runs', 'total_wickets','batsman_average','batsman_strike_rate','bowling_economy','bowler_strike_rate','striker_runs', 'striker_balls', 'ball_number']])
    output_data= np.array(final_data[['runs_0','runs_1','runs_2','runs_3','runs_4','runs_6','is_out']])
    return input_data,output_data

In [7]:
train_x,train_y = convert_data(data)

In [8]:
train_y.shape

(88995, 7)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)

In [10]:
full_train_x = train_x
full_train_y = train_y

In [11]:
#splitting
test_x = train_x[-10000:,:]
test_y = train_y[-10000:,:]
train_x = train_x[0:-10000,:]
train_y = train_y[0:-10000,:]

In [12]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier().fit(train_x,train_y) 

pred_y = dtree_model.predict(test_x)
from sklearn.metrics import accuracy_score
print("The accuracy when using decision tree classifier is ",accuracy_score(test_y,pred_y))

The accuracy when using decision tree classifier is  0.3008


In [13]:
test_y

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
import keras
model = Sequential()
model.add(Dense(32, input_dim=9, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(16))
model.add(Dense(16))
model.add(Dense(7, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [15]:
train_x.shape

(78995, 9)

In [16]:
test_y.shape

(10000, 7)

In [17]:
history  = model.fit(train_x,train_y,batch_size=1, epochs=100,validation_data=(test_x,test_y))

Epoch 1/100
78995/78995 [==============================] - 144s 2ms/step - loss: 0.3328 - accuracy: 0.4587 - val_loss: 0.3251 - val_accuracy: 0.4551
Epoch 2/100
78995/78995 [==============================] - 143s 2ms/step - loss: 0.3242 - accuracy: 0.4700 - val_loss: 0.3235 - val_accuracy: 0.4661
Epoch 3/100
78995/78995 [==============================] - 146s 2ms/step - loss: 0.3208 - accuracy: 0.4704 - val_loss: 0.3235 - val_accuracy: 0.4598
Epoch 4/100
78995/78995 [==============================] - 146s 2ms/step - loss: 0.3214 - accuracy: 0.4737 - val_loss: 0.3234 - val_accuracy: 0.4615
Epoch 5/100
78995/78995 [==============================] - 148s 2ms/step - loss: 0.3222 - accuracy: 0.4693 - val_loss: 0.3236 - val_accuracy: 0.4610
Epoch 6/100
78995/78995 [==============================] - 147s 2ms/step - loss: 0.3207 - accuracy: 0.4721 - val_loss: 0.3228 - val_accuracy: 0.4594
Epoch 7/100
78995/78995 [==============================] - 146s 2ms/step - loss: 0.3221 - accuracy: 0.4702

In [18]:
def onehot_to_indices(dataset):
    ans = []
    for i in range(dataset.shape[0]):
        a = False
        for j in range(dataset[i].shape[0]):
            if(dataset[i][j]==1 and not a):
                ans.append(j+1)
                a = True
        if(not a):
            ans.append(1)
    return ans
    

In [19]:
series = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4 AND start_date BETWEEN '01-01-2020 00:00:01' AND '12-12-2020 00:00:01'", conn)
query = f"""SELECT id FROM cds_match_match WHERE series_id = {int(series['id'])} """
match_ids = pd.read_sql_query(query,conn)
query = f"""SELECT id,match_id FROM cds_scorecard_scorecard WHERE match_id in {tuple(match_ids['id'])} and inning = 1 """
scorecard_ids = pd.read_sql_query(query,conn)
query = f"""SELECT * FROM cds_commentary_commentary WHERE scorecard_id in {tuple(scorecard_ids['id'])} """
test_data = pd.read_sql_query(query,conn)
test_data = test_data[['id', 'ball', 'over', 'total_runs', 'total_wickets','is_wide', 'runs', 'is_out', 'is_no_ball', 'batsmen_id', 'bowler_id', 'offstrike_batsmen_id','player_out_id', 'scorecard_id', 'team_id']]
scorecard_ids.columns = ['scorecard_id', 'match_id']
test_data = test_data.merge(scorecard_ids, on='scorecard_id', how='left')
test_data = test_data.sort_values(['scorecard_id','over','ball'])

In [20]:
test_X,test_Y = convert_data(test_data)
test_X = scaler.transform(test_X)

In [21]:
predictions = model.predict(test_X)

In [22]:
def softmm(predictions):
    for i in range(len(predictions)):
        index = list(predictions[i]).index(max(list(predictions[i])))
        for j in range(7):
            if(index == j):
                predictions[i][j]=1
            else:
                predictions[i][j]=0
    return predictions

from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier().fit(full_train_x,full_train_y) 
predictions = dtree_model.predict(test_X)
from sklearn.metrics import accuracy_score
accuracy_score(test_Y,predictions)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
NBclassifier = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
NBclassifier.fit(train_x, train_y)

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
import keras
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(32, input_dim=9, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(7,activation = 'softmax'))
opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)
#model.compile(optimizer=keras.optimizers.Adam(lr = 0.001,clipvalue = 1),loss=keras.losses.CategoricalCrossentropy(),metrics=[keras.metrics.CategoricalAccuracy()])
#model.summary()
print("Fit model on training data")
history = model.fit(
    train_x,
    train_y,
    batch_size=64,
    epochs=10,
    validation_data=(val_x,val_y),
)